In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
import plotly.express as px

 **1. Carga y Exploración de Datos:**

In [ ]:
# Usamos la biblioteca google para poder usar archivos en nuestro drive.
from google.colab import drive
# Este comando conecta colab con drive.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/datasets/vehicles.csv"
df = pd.read_csv(path)

**2. LIMPIEZA Y PREPROCESAMIENTO**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [ ]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


**3. EXPLORACION DE DATOS**

In [ ]:
# Ver cuántos ids únicos hay
print("IDs únicos:", df["id"].nunique())
print("Total de filas:", len(df))

IDs únicos: 426880
Total de filas: 426880


In [ ]:
# Porcentaje de nulos por columna
porcentaje_nulos = df.isna().mean().sort_values(ascending=False) * 100

# Mostrar en forma de tabla
porcentaje_nulos = porcentaje_nulos.round(2).reset_index()
porcentaje_nulos.columns = ["Columna", "Porcentaje de Nulos"]
porcentaje_nulos

,Columna,Porcentaje de Nulos
0,county,100.00
1,size,71.77
2,cylinders,41.62
3,condition,40.79
4,VIN,37.73
5,drive,30.59
6,paint_color,30.50
7,type,21.75
8,manufacturer,4.13
9,title_status,1.93


In [ ]:
df["url"]

,url
0,https://prescott.craigslist.org/cto/d/prescott...
1,https://fayar.craigslist.org/ctd/d/bentonville...
2,https://keys.craigslist.org/cto/d/summerland-k...
3,https://worcester.craigslist.org/cto/d/west-br...
4,https://greensboro.craigslist.org/cto/d/trinit...
...,...
426875,https://wyoming.craigslist.org/ctd/d/atlanta-2...
426876,https://wyoming.craigslist.org/ctd/d/atlanta-2...
426877,https://wyoming.craigslist.org/ctd/d/atlanta-2...
426878,https://wyoming.craigslist.org/ctd/d/atlanta-2...


In [ ]:
df["region"]

,region
0,prescott
1,fayetteville
2,florida keys
3,worcester / central MA
4,greensboro
...,...
426875,wyoming
426876,wyoming
426877,wyoming
426878,wyoming


In [ ]:
df["region_url"]

,region_url
0,https://prescott.craigslist.org
1,https://fayar.craigslist.org
2,https://keys.craigslist.org
3,https://worcester.craigslist.org
4,https://greensboro.craigslist.org
...,...
426875,https://wyoming.craigslist.org
426876,https://wyoming.craigslist.org
426877,https://wyoming.craigslist.org
426878,https://wyoming.craigslist.org


In [ ]:
df["VIN"]

,VIN
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
426875,1N4AA6AV6KC367801
426876,7JR102FKXLG042696
426877,1GYFZFR46LF088296
426878,58ABK1GG4JU103853


In [ ]:
df["size"]

,size
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
426875,NaN
426876,NaN
426877,NaN
426878,NaN


In [ ]:
df["type"]

,type
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
426875,sedan
426876,sedan
426877,hatchback
426878,sedan


In [ ]:
df["image_url"]

,image_url
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
426875,https://images.craigslist.org/00o0o_iiraFnHg8q...
426876,https://images.craigslist.org/00x0x_15sbgnxCIS...
426877,https://images.craigslist.org/00L0L_farM7bxnxR...
426878,https://images.craigslist.org/00z0z_bKnIVGLkDT...


In [ ]:
df["description"]

,description
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
426875,Carvana is the safer way to buy a car During t...
426876,Carvana is the safer way to buy a car During t...
426877,Carvana is the safer way to buy a car During t...
426878,Carvana is the safer way to buy a car During t...


In [ ]:
df["posting_date"]

,posting_date
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
426875,2021-04-04T03:21:31-0600
426876,2021-04-04T03:21:29-0600
426877,2021-04-04T03:21:17-0600
426878,2021-04-04T03:21:11-0600


Analizando las varibles anteriores de tipo texto y algunas númericas, además del porcentaje de valores nulos que poseen, procedo a eliminar las variables:

id, url, region_url, VIN, image_url y description, porque las considero pocos utiles en el analisis posterior, algunas son textos y otras son tipo codigos.

Tambien eliminare county y size, ya que poseen un gran porcentaje de valores nulos respecto al total, 100% y 71.77% respectivamente.

De igual forma eliminare posting_date ya que contiene fecha y hora, y no es relevante en el analisis posterior.

In [ ]:
# Eliminacion de columnas irrelevantes para el analisis, y con gran porcentaje de valores nulos.
columnas_a_eliminar = ["id", "url", "region_url", "VIN", "image_url", "description", "county", "size","posting_date"]

df.drop(columns=columnas_a_eliminar, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        426880 non-null  object 
 1   price         426880 non-null  int64  
 2   year          425675 non-null  float64
 3   manufacturer  409234 non-null  object 
 4   model         421603 non-null  object 
 5   condition     252776 non-null  object 
 6   cylinders     249202 non-null  object 
 7   fuel          423867 non-null  object 
 8   odometer      422480 non-null  float64
 9   title_status  418638 non-null  object 
 10  transmission  424324 non-null  object 
 11  drive         296313 non-null  object 
 12  type          334022 non-null  object 
 13  paint_color   296677 non-null  object 
 14  state         426880 non-null  object 
 15  lat           420331 non-null  float64
 16  long          420331 non-null  float64
dtypes: float64(4), int64(1), object(12)
memory usage

In [ ]:
df["lat"]

,lat
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
426875,33.786500
426876,33.786500
426877,33.779214
426878,33.786500


In [ ]:
df["lat"]

,long
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
426875,-84.445400
426876,-84.445400
426877,-84.411811
426878,-84.445400


Lat y long representan la ubicacion, ademas a ambas les faltan los mismos valores, no las considero esenciales para el analisis porque son coordenadas exactas y para la ubicacion cuento con region y state

In [ ]:
# Eliminacion de columnas irrelevantes
columnas_a_eliminar2 = ["lat", "long"]

df.drop(columns=columnas_a_eliminar2, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        426880 non-null  object 
 1   price         426880 non-null  int64  
 2   year          425675 non-null  float64
 3   manufacturer  409234 non-null  object 
 4   model         421603 non-null  object 
 5   condition     252776 non-null  object 
 6   cylinders     249202 non-null  object 
 7   fuel          423867 non-null  object 
 8   odometer      422480 non-null  float64
 9   title_status  418638 non-null  object 
 10  transmission  424324 non-null  object 
 11  drive         296313 non-null  object 
 12  type          334022 non-null  object 
 13  paint_color   296677 non-null  object 
 14  state         426880 non-null  object 
dtypes: float64(2), int64(1), object(12)
memory usage: 48.9+ MB


In [ ]:
# Analizar estadisticas basicas
df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
price,426880.0,75199.03,12182282.17,0.0,5900.0,13950.0,26485.75,3.736929e+09
year,425675.0,2011.24,9.45,1900.0,2008.0,2013.0,2017.00,2.022000e+03
odometer,422480.0,98043.33,213881.50,0.0,37704.0,85548.0,133542.50,1.000000e+07


analizando las estadisticas basicas:

En price el maximo es muy elevado, mas adelante lo analizare, posible outlier.
En year parece estar todo bien, año 1900 podrian ser autos clasicos, ademas los valores faltantes que hay los reemplazare por la mediana que es 2013 lo cual es mas representativa que la media.
En odometer hay valores extremos igual, posibles outliers, mas adelante los analizo.

In [ ]:
# Ver los 10 valores extremos en price
df[["price"]].sort_values(by="price", ascending=False).head(10)

,price
80181,99999
255553,99999
246133,99999
277151,99999
44134,99999
44389,99999
77605,99999
363662,99995
369050,99995
378383,99995


In [ ]:
df[df["price"] > 100000].shape[0]

0

In [ ]:
df[df["price"] > 1000000].shape[0]

0

In [ ]:
df[df["price"] > 500000].shape[0]

0

In [ ]:
df[df["price"] > 100000].shape[0]

0

In [ ]:
df[df["price"] > 1000000].shape[0]
df[df["price"] > 500000].shape[0]
df[df["price"] > 100000].shape[0]

655

In [ ]:
# Ver los 10 valores extremos en odometer
df[["odometer"]].sort_values(by="odometer", ascending=False).head(10)

,odometer
320410,10000000.0
110623,10000000.0
103798,10000000.0
108102,10000000.0
105059,10000000.0
346469,10000000.0
262338,10000000.0
9218,10000000.0
144436,10000000.0
413267,10000000.0


Con el dataframe sin duplicados y con solo las columnas utiles para el analisis, procedo a imputar los valores faltantes.

**4. MODELADO Y EVALUACION:**

**5. OPTIMIZACION DEL MODELO:**

**CONCLUSION**